<div style="border:solid green 2px; padding: 20px">
<b>Привет, Мария!</b>

Меня зовут Александр Пономаренко, и я буду проверять твой проект. Предлагаю общаться на «ты» :) Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать data science. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - **пожалуйста, не перемещай, не изменяй и не удаляй их**. Увидев у тебя ошибку, в первый раз я лишь укажу на ее наличие и дам тебе возможность самой найти и исправить ее. На реальной работе твой начальник будет поступать так же, а я пытаюсь подготовить тебя именно к работе аналитиком. Но если ты пока не справишься с такой задачей - при следующей проверке я дам более точную подсказку. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера ❌:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️:</b> Так выделены небольшие замечания. Я надеюсь, что их ты тоже учтешь - твой проект от этого станет только лучше. Но настаивать на их отработке не буду.

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: **если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом.** Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

<div class="alert alert-info"> <b>Комментарий студента:</b> Спасибо за рекомендации! Я переписала короче многие блоки. Буду рада рекомендациям по стилю кода :)</div>

# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

In [1]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import isnan

from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

RANDOM_SEED = 2022

Инициализация spark-сессии:

In [2]:
spark = SparkSession.builder \
                    .master('local') \
                    .appName('Prediction house value')  \
                    .getOrCreate()

# Подготовка данных

Чтение данных:

In [3]:
df = spark.read.load('/datasets/housing.csv', format='csv', sep=',', inferSchema=True, header='true')

df.printSchema()
df.toPandas().head()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отлично, данные на месте:)</div>

Просмотр базовых описательных статистик:

In [4]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Исследование данных на наличие пропусков:

In [5]:
def count_null(df):
    for col in df.columns:
        print(col, df.filter(F.col(col).isNull() | isnan(F.col(col))).count()/df.count())

In [6]:
count_null(df)

longitude 0.0
latitude 0.0
housing_median_age 0.0
total_rooms 0.0
total_bedrooms 0.01002906976744186
population 0.0
households 0.0
median_income 0.0
median_house_value 0.0
ocean_proximity 0.0


В столбце `total_bedrooms` 1% пропусков. Заполним их медианными значениями:

In [7]:
df = Imputer(inputCol='total_bedrooms', outputCol='total_bedrooms_imputed', strategy='median') \
        .fit(df) \
        .transform(df)

count_null(df)

longitude 0.0
latitude 0.0
housing_median_age 0.0
total_rooms 0.0
total_bedrooms 0.01002906976744186
population 0.0
households 0.0
median_income 0.0
median_house_value 0.0
ocean_proximity 0.0
total_bedrooms_imputed 0.0


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Хорошо:) Как вариант можно заполнить вот так:

```python
    imputer = Imputer(inputCol='total_bedrooms', outputCol="out_total_bedrooms", strategy='median')
    model = imputer.fit(df)
    new_df = model.transform(df)
    
```


</div>


<div class="alert alert-info"> <b>Комментарий студента:</b> Заполнила пропуски с помощью Imputer.</div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Супер:)


</div>


Пропуски заполнены.

Преобразование столбцов с категориальными значениями техникой OHE:

In [8]:
categorical_cols = ['ocean_proximity']
numerical_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms_imputed',
                  'population', 'households', 'median_income']
target = 'median_house_value'

In [9]:
df = StringIndexer(inputCols=categorical_cols, 
                   outputCols=[c+'_idx' for c in categorical_cols]) \
                        .fit(df) \
                        .transform(df)

df = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                   outputCols=[c+'_ohe' for c in categorical_cols]) \
                        .fit(df) \
                        .transform(df)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
df.select(cols).show(5)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 5 rows



<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Супер, данные закодированы;) 

</div>


Шкалирование числовых признаков:

In [10]:
df = VectorAssembler(inputCols=numerical_cols, outputCol='numerical_features') \
                        .transform(df)

df = StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled') \
                        .fit(df) \
                        .transform(df)

df.select('numerical_features', 'numerical_features_scaled').show(3)

+--------------------+-------------------------+
|  numerical_features|numerical_features_scaled|
+--------------------+-------------------------+
|[-122.23,37.88,41...|     [-61.007269596069...|
|[-122.22,37.86,21...|     [-61.002278409814...|
|[-122.24,37.85,52...|     [-61.012260782324...|
+--------------------+-------------------------+
only showing top 3 rows



<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отмасштабировала корректно:) 

</div>


Объединение категориальных признаков в один вектор:

In [11]:
df = VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                     outputCol='categorical_features') \
                         .transform(df)

df.select('categorical_features').show(3)

+--------------------+
|categorical_features|
+--------------------+
|       (4,[3],[1.0])|
|       (4,[3],[1.0])|
|       (4,[3],[1.0])|
+--------------------+
only showing top 3 rows



Объединение трансформированных категориальных и числовых признаков в один вектор:

In [12]:
df = VectorAssembler(inputCols=['categorical_features', 'numerical_features_scaled'],
                     outputCol='features') \
                          .transform(df)

df.select('features').show(3)

+--------------------+
|            features|
+--------------------+
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
+--------------------+
only showing top 3 rows



# Обучение моделей

Разделение датасета на выборки:

In [13]:
df_train, df_test = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(df_train.count(), df_test.count())

16418 4222


Функции:
- вывода метрик модели
- перебора гиперпараметров для нахождения модели с наилучшей метрикой

In [14]:
def print_metrics(evaluators, predictions):
    for evaluator in evaluators:
        print(evaluator, evaluators[evaluator].evaluate(predictions))

def cross_validate(lr, metric):
    param_grid = ParamGridBuilder().addGrid(lr.regParam, [0.01, 0.5, 2.0]) \
                                   .addGrid(lr.maxIter, [1, 5, 10, 20, 50]) \
                                   .build()
    
    return TrainValidationSplit(estimator=lr,
                                estimatorParamMaps=param_grid,
                                evaluator=metric,
                                trainRatio=0.8,
                                seed=RANDOM_SEED) \
                                    .fit(df_train)

In [15]:
metrics ={}
for metric in ['rmse', 'mae', 'r2']:
    metrics[metric] = RegressionEvaluator(labelCol=target, metricName=metric)

Построение модели, используя все признаки (числовые и категориальные):

In [16]:
predictions_on_full_data = cross_validate(LinearRegression(labelCol=target, featuresCol='features'), 
                                          metrics['r2']) \
                                              .transform(df_test)

predictions_on_full_data.select(target, 'prediction').show(20)

22/12/06 13:07:58 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/12/06 13:07:58 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/12/06 13:07:58 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/12/06 13:07:58 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|153022.45945984684|
|           50800.0| 214986.2133011222|
|           58100.0|142730.73453220492|
|           68400.0| 132516.4871437042|
|           72200.0| 164118.0685549113|
|           67000.0| 154462.2806498059|
|           81300.0|152888.52964031044|
|           70500.0|164598.49198417598|
|           60000.0|142753.25977558177|
|          109400.0|171347.00868791295|
|           74100.0|150635.48165183095|
|           74700.0|167891.84043503273|
|           90000.0| 209863.1348652253|
|          104200.0| 200519.7420377531|
|           74100.0|157172.42768315692|
|           67500.0| 148339.8283128054|
|          103100.0| 47490.49748268956|
|           92500.0|  166748.314372533|
|          128100.0|222114.80621313443|
|           99600.0|187291.90475222096|
+------------------+------------------+
only showing top 20 rows



Построение модели, используя только числовые признаки:

In [17]:
predictions_on_numerical_data = cross_validate(LinearRegression(labelCol=target, featuresCol='numerical_features_scaled'),
                                               metrics['r2']) \
                                                   .transform(df_test)

predictions_on_numerical_data.select(target, 'prediction').show(20)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0| 101406.2913245773|
|           50800.0|183331.75189457834|
|           58100.0|109608.81756080175|
|           68400.0| 80444.21502781706|
|           72200.0|130001.38464584807|
|           67000.0|120453.19644093327|
|           81300.0|118117.53252915805|
|           70500.0|130758.30230018031|
|           60000.0|110065.79480161844|
|          109400.0|118254.46115862159|
|           74100.0|117750.80850355932|
|           74700.0|134467.25967075862|
|           90000.0| 176033.3946645148|
|          104200.0|166605.98543671798|
|           74100.0|123050.17657577479|
|           67500.0|113817.08334864723|
|          103100.0| -8380.06678610947|
|           92500.0| 140926.1828873302|
|          128100.0|191274.31017359626|
|           99600.0|152720.48398614628|
+------------------+------------------+
only showing top 20 rows



<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отлично, обучила верно:) 

</div>


# Анализ результатов

Сравним метрики RMSE, MAE и R2 моделей, обучающихся на разных наборах данных.

Метрики модели, обучающейся на всех признаках (числовых и категориальных):

In [18]:
print_metrics(metrics, predictions_on_full_data)

rmse 68486.92403262584
mae 49866.91155597181
r2 0.6535562575837126


Метрики модели, обучающейся только на числовых признаках:

In [19]:
print_metrics(metrics, predictions_on_numerical_data)

rmse 69212.85461378017
mae 50866.422500750974
r2 0.6461730388734785


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> протестировала корретно:) 
    
Пример подбора параметров:)    
```python
    gtb = GBTRegressor(featuresCol = 'features', labelCol='median_house_value')
    paramGrid = ParamGridBuilder().addGrid(gtb.maxIter, <your code>)\
                                  .addGrid(gtb.maxDepth , <your code>)\
                                  .addGrid(gtb.stepSize , <your code>)\
                                  .build()
    evaluator = RegressionEvaluator(labelCol='median_house_value',metricName='r2')
    valid = TrainValidationSplit(estimator=gtb,
                                 estimatorParamMaps=paramGrid,
                                 evaluator=evaluator,
                                 trainRatio=0.8,
                                 seed=RANDOM_SEED)
```


</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Добавила подбор параметров. TrainValidationSplit перебирает параметры для получения модели с наилучшей метрикой. Мне надо сравнить модели по 3 метрикам, а валидирую я по одной (r2 - выбрана наугад из трех). Какую метрику лучше передавать в TrainValidationSplit в evaluator?
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Все хорошо:) Думаю, тут можно любую, не принципиально


</div>

In [20]:
spark.stop()

Выводы:
- RMSE меньше у модели, обучающейся на всех данных;
- MAE меньше у модели, обучающейся на всех данных;
- R2 больше у модели, обучающейся на всех данных;

У модели, обучившейся на всех данных (числовых и категориальных), метрики RMSE, MAE, R2 лучше по сравнению с моделью, обучившейся только на числовых данных.

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>  Немного полезного материала:

Немного полезной информации:
+  https://www.tutorialspoint.com/pyspark/index.htm
+  https://www.guru99.com/pyspark-tutorial.html
+  https://databricks.com/spark/getting-started-with-apache-spark/machine-learning#load-sample-data


</div>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>
<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>Мария, спасибо за хороший проект!!! Я готов принять работу, но хочу убедиться, что тебе все понятно.<br>
Если есть какие либо вопросы я с удовольствием на них отвечу:)</div>


<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Удачи в следующих проектах!!!


</div>